Containers are package of software help in
- Development
- Deployment

Bare Metal vs VM vs Container

Assume we are building app with stack
- Web server = Node Express
- DB = Mongo
- Messaging = Redis
- Orchestration = Ansible
- We need different envs for test/prod/dev

Bare Metal
- Dependency conflicts
- Long Setup time
- High Blast radius
- 

VM are of two types
![vm-arch](images/vm-architechture.png "VM -ARCH")
- Type 1 VM = Dual Boot (Hyper V)
- Type 2 VM = Booted over an underlying OS (Virtual Box)
- No dependency conflicts
- Complete isolation
- Hypervisor

Container
![container-arch](images/container-architechture.png "Container-ARCH")
- Container Runtimes
- Low Utilization, Fast Boot, Less Disk Space

Hybrid Arch
![mix](images/container-over-vms.png "Hybrid Arch")
- Containers over VMs


OCI Specifications
- Runtime
- Image
- Distribution

Lingo
- DockerHub = Public registry
- DockerFile = Commands to create an Image
- Docker Image = Class
- Docker Container = Objects of image
- Docker container only runs while the process inside is active. It exits when the process exits.

Commands

1. docker run [image]
- Pull image if not present and Runs the container
- Options
    1. -d
        - detached mode
        - returns the container id and runs in bg
        - To bring to foreground run ```docker attach [container_id]```
    2. -it
        - -i = interactive mode
        - -t = pseudoterminal
        - -it = interactive and pseudoterminal mode
    3. -e
        - run container with environmental variables
        - To get list of all vars run ```docker inspect [container]```
    4. -p
        - Port mapping
        - Map Host port to container port
        - ![docker-port-mapping](images/docker-port-mapping.png)
        - docker run -p [host-port]:[container-port] [container]
    5. -v
        - volume mapping
        - map host storage volume with container storage volume
        - data loss prevention
        - ![vol-port](images/docker-volume-mapping.png)
        - docker run -v [host-vol]:[cont-vol] [container]


2. docker pull [image]
- Pull image if not present. No Run.

3. docker ps
- list running containers
 
docker inspect [container]
docker logs [container]

4. docker ps -a
- list all containers

docker stop [container]
docker rm [container]
docker images
- lists all pulled images

docker rmi [image]

docker run ubuntu sleep 5
docker exec [container] [command]

- Useful for debugging
- docker exec -it [container] /bin/bash
- docker logs [contianer-id]

run -vs- start
on images -- on containers
new container -- restart existing container

 



# Creating Images
1. Create Dockerfile
2. Build Dockerfile
3. Push to Docker Registry

Sample Dockerfile
```docker
FROM Ubuntu
ENV EK1=EV1 EK2=EV2
RUN apt update
RUN apt install python
RUN pip install flask
COPY . /cont/source
CMD ["app.py", "flask", "run"] (or)
ENTRYPOINT FLASK_APP=/cont/source/app.py flask run
```

```
docker build -t your-image:1.0 .
docker push acc_name/image_name
```

- Workflow on editing the Dockerfile
```
docker stop container
docker rm container
docker rmi image
docker build -t image:1.0 Dockerfile
docker run image
```

1. Layered Architechture
2. all layers are cached
3. docker history acc_name/image_name

- Specify different command as entrypoint
- ENTRYPOINT ["sleep"]
- CMD ["5"]

# Networking in Docker
```docker network ls```
```docker network create [network-name]```
```docker run --net=[network-name] image```
1. Bridge
    - default
    - 172.x
    - Containers can access each other over this bridge network
    - It only creates one bridge by default but we can create a custom bridge by
    - ```
      docker network create --driver bridge \
      --subnet 182.18.0.0/16
      custom-isolated-network
      ``` 
2. None
    - --network=none
    - runs in isolated network
3. Host
    - port mapping not requires


Docker DNS server
- Default 127.0.0.11
- Resolves container names to their IP

# File Storage
```var/lib/docker```
- Files in **Image Layers** are immutable
- Files in **Container Layers** are mutable until the container is active
1. Copy on Write: Editing files of image layer
    - Docker copies files from image layer to container layer
    - Changes are active only while the container is active.
2. Volume Mounting: Container Storage
    - docker volume create vol_name
    - docker run -v vol_name:/var/lib/mysql mysql
3. Bind Mounting: Host Storage
    - docker run --mount type=bind,source=host/vol,target=cont/vol container_name

# Docker Compose Config Files
```docker-compose -d -f c.yaml up/down```

Example App you need:
1. Redis: ```docker run -d --name=redisdb redis```
2. Postgres: ```docker run -d --name=postgresdb postgres```
3. Fastapi: ```docker run -d --name=mainapp -p 5000:80 --link redis:redisdb fastapiapp```

> Compose file
```
<name>:
    image: <image>
    build: <path>
    ports:
        - <port1>:<port2>
    links:
        - <name1>
        - <name2>
```

```
version: 2
services:
    <name>:
        image: <image>
        build: <path>
        ports:
            - <port1>:<port2>
        depends_on:
            - <name1>
            - <name2>
```

```swarm.yaml
version: 3
services:
    <name>:
        image: <image>
        build: <path>
        ports:
            - <port1>:<port2>
        environment:
            - K1=V1
            - K2=V2
        depends_on:
            - <name1>
            - <name2>
networks:
    front-end:
    back-end:
```

# Registry

image: registry/user/image
example: docker.io/nginx/nginx

Private registry
- docker login private.io
- docker run private.io/acc/private-image

Registry itself is a container and can be used to host your private images
image: registry

1. docker run -d --name=registry -p 5000:5000 registry
2. docker image tag my-image localhost:5000/my-image
3. docker push localhost:5000/my-image
4. docker pull localhost:5000/my-image

# Engine

Remotely run containers on server
```docker -H=192.168.1.123 run image```
```docker --cpu=0.5 --memory=100m run image```

# Ochestration: k8s













